
# Simple To-Do CLI app (no external libraries)
Features:
- Add, list, edit, mark complete, delete tasks
- Save to / Load from a JSON file
- Autosave after changes (configurable)
- Robust input handling and polite messages


In [ ]:
import json
import os
from datetime import datetime

DATA_FILE = "todo_data.json"   # default filename for saving/loading tasks
AUTOSAVE = True                # if True, saves after every change

# -------------------- Data Handling --------------------
def load_tasks(filename=DATA_FILE):
    if not os.path.exists(filename):
        return []
    try:
        with open(filename, "r", encoding="utf-8") as f:
            data = json.load(f)
            if isinstance(data, list):
                return data
            else:
                print("Warning: data file malformed, starting with empty list.")
                return []
    except (json.JSONDecodeError, IOError) as e:
        print("Warning: failed to load tasks:", e)
        return []

def save_tasks(tasks, filename=DATA_FILE):
    try:
        with open(filename, "w", encoding="utf-8") as f:
            json.dump(tasks, f, ensure_ascii=False, indent=2)
    except IOError as e:
        print("Error saving tasks:", e)

def next_id(tasks):
    if not tasks:
        return 1
    return max(task.get("id", 0) for task in tasks) + 1

# -------------------- Task Actions --------------------
def add_task(tasks):
    title = input("Task title: ").strip()
    if not title:
        print("Cancelled: title cannot be empty.")
        return
    notes = input("Notes (optional): ").strip()
    due_input = input("Due date (YYYY-MM-DD) (optional): ").strip()
    due = None
    if due_input:
        try:
            due_dt = datetime.strptime(due_input, "%Y-%m-%d")
            due = due_dt.date().isoformat()
        except ValueError:
            print("Invalid date format. Skipping due date.")

    try:
        pr = int(input("Priority 1 (high) - 5 (low) [default 3]: ").strip() or "3")
        if pr < 1 or pr > 5:
            raise ValueError
    except ValueError:
        print("Invalid priority, using default 3.")
        pr = 3

    task = {
        "id": next_id(tasks),
        "title": title,
        "notes": notes,
        "created_at": datetime.utcnow().isoformat(),
        "due": due,
        "priority": pr,
        "done": False
    }
    tasks.append(task)
    print(f"Added task #{task['id']}: {task['title']}")
    if AUTOSAVE:
        save_tasks(tasks)

def format_task_line(task, index=None):
    idx_part = f"{index+1:2d}." if index is not None else ""
    status = "✓" if task.get("done") else " "
    due = task.get("due") or "-"
    pr = task.get("priority", 3)
    return f"{idx_part} [{status}] (id:{task['id']}) P{pr} Due:{due} — {task['title']}"

def list_tasks(tasks, show_all=True):
    if not tasks:
        print("No tasks found.")
        return
    def sort_key(t):
        return (t.get("done", False), t.get("priority", 3), t.get("due") or "")
    sorted_tasks = sorted(tasks, key=sort_key)
    for i, t in enumerate(sorted_tasks):
        if not show_all and t.get("done"):
            continue
        print(format_task_line(t, index=i))
        if t.get("notes"):
            print("     Notes:", t["notes"])

def find_task_by_id(tasks, id_value):
    for t in tasks:
        if t.get("id") == id_value:
            return t
    return None

def mark_done(tasks):
    try:
        id_input = int(input("Enter task id to mark as done: ").strip())
    except ValueError:
        print("Invalid id.")
        return
    task = find_task_by_id(tasks, id_input)
    if not task:
        print("Task not found.")
        return
    if task.get("done"):
        print("Task is already marked done.")
        return
    task["done"] = True
    print(f"Task #{task['id']} marked as done.")
    if AUTOSAVE:
        save_tasks(tasks)

def delete_task(tasks):
    try:
        id_input = int(input("Enter task id to delete: ").strip())
    except ValueError:
        print("Invalid id.")
        return
    task = find_task_by_id(tasks, id_input)
    if not task:
        print("Task not found.")
        return
    confirm = input(f"Are you sure you want to delete #{task['id']} '{task['title']}'? (y/N): ").strip().lower()
    if confirm == "y":
        tasks.remove(task)
        print("Task deleted.")
        if AUTOSAVE:
            save_tasks(tasks)
    else:
        print("Deletion cancelled.")

def edit_task(tasks):
    try:
        id_input = int(input("Enter task id to edit: ").strip())
    except ValueError:
        print("Invalid id.")
        return
    task = find_task_by_id(tasks, id_input)
    if not task:
        print("Task not found.")
        return
    print("Leave input empty to keep current value.")
    new_title = input(f"Title [{task['title']}]: ").strip()
    if new_title:
        task['title'] = new_title
    new_notes = input(f"Notes [{task.get('notes','')}]: ").strip()
    if new_notes:
        task['notes'] = new_notes
    new_due = input(f"Due [{task.get('due','')}]: ").strip()
    if new_due:
        try:
            datetime.strptime(new_due, "%Y-%m-%d")
            task['due'] = new_due
        except ValueError:
            print("Invalid date format. Keeping old due date.")
    try:
        new_pr = input(f"Priority [{task.get('priority',3)}]: ").strip()
        if new_pr:
            pr = int(new_pr)
            if 1 <= pr <= 5:
                task['priority'] = pr
            else:
                print("Priority must be 1-5. Keeping old priority.")
    except ValueError:
        print("Invalid priority input. Keeping old priority.")
    print("Task updated.")
    if AUTOSAVE:
        save_tasks(tasks)

# -------------------- Save/Load Commands --------------------
def save_command(tasks):
    filename = input(f"Filename [{DATA_FILE}]: ").strip() or DATA_FILE
    save_tasks(tasks, filename)
    print("Saved to", filename)

def load_command():
    filename = input(f"Filename [{DATA_FILE}]: ").strip() or DATA_FILE
    tasks = load_tasks(filename)
    print(f"Loaded {len(tasks)} tasks from {filename}.")
    return tasks

# -------------------- Help Menu --------------------
def show_help():
    print("""
Commands:
  add      - add a new task
  list     - list all tasks
  list-p   - list pending tasks only
  done     - mark a task done by id
  edit     - edit a task by id
  del      - delete a task by id
  save     - save tasks to file
  load     - load tasks from file (replaces current in-memory list)
  help     - show this help
  exit     - quit the program
""")

# -------------------- Main Loop --------------------
def main_loop():
    tasks = load_tasks()
    print("Welcome to the To-Do CLI. Type 'help' for commands.\n")
    while True:
        cmd = input("> ").strip().lower()
        if cmd in ("add", "a"):
            add_task(tasks)
        elif cmd in ("list", "ls"):
            list_tasks(tasks, show_all=True)
        elif cmd in ("list-p", "pending"):
            list_tasks(tasks, show_all=False)
        elif cmd in ("done",):
            mark_done(tasks)
        elif cmd in ("del", "delete"):
            delete_task(tasks)
        elif cmd in ("edit",):
            edit_task(tasks)
        elif cmd in ("save",):
            save_command(tasks)
        elif cmd in ("load",):
            tasks = load_command()
        elif cmd in ("help", "?"):
            show_help()
        elif cmd in ("exit", "quit"):
            if not AUTOSAVE:
                confirm = input("Save before exit? (Y/n): ").strip().lower()
                if confirm != "n":
                    save_tasks(tasks)
            print("Goodbye.")
            break
        elif cmd == "":
            continue
        else:
            print("Unknown command. Type 'help' for list of commands.")

if __name__ == "__main__":
    main_loop()



Welcome to the To-Do CLI. Type 'help' for commands.


Commands:
  add      - add a new task
  list     - list all tasks
  list-p   - list pending tasks only
  done     - mark a task done by id
  edit     - edit a task by id
  del      - delete a task by id
  save     - save tasks to file
  load     - load tasks from file (replaces current in-memory list)
  help     - show this help
  exit     - quit the program

